A series of tests of the SLIP package.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
fopts = {'fontsize':18}

In [ ]:
from SLIP import Image, imread

In [ ]:
help(Image)

## initializing the SLIP object

The SLIP object may be initialized by giving some parameters. These could be given using 

- nothing

In [ ]:
im = Image()  # default is 'https://raw.githubusercontent.com/bicv/SLIP/master/default_param.py'
print(im.pe)

- a dictionary (dimensions ``N_X`` and ``N_Y`` and ``mask_exponent`` are mandatory)

In [ ]:
im = Image({'N_X':32, 'N_Y':64})
print(im.pe)

- a ``ndarray`` (dimensions ``N_X`` and ``N_Y`` are guessed from this array)

In [ ]:
im = Image(np.empty((32, 64)))
print(im.pe)

- a string indicating the path of a image file

In [ ]:
im = Image('database/lena64.png')
print(im.pe)

- a string indicating the URL to a image file

In [ ]:
im = Image('http://upload.wikimedia.org/wikipedia/commons/d/de/Wikipedia_Logo_1.0.png')
print(im.pe)

- a string indicating the path of a file containing a dictionary

In [ ]:
im = Image('file://default_param.py')
im = Image('default_param.py')
print(im.pe.keys())

- a string indicating the URL of a file containing a dictionary

In [ ]:
im = Image('https://raw.githubusercontent.com/bicv/SLIP/master/default_param.py')
print(im.pe.keys())

- a NeuroTools object

In [ ]:
from NeuroTools.parameters import ParameterSet
from SLIP import Image
im = Image(ParameterSet({'N_X':128, 'N_Y':256}))

The parameters may be handled using the properties of the ``ParameterSet`` object.

## resizing the SLIP object

The SLIP object may be adapted to a new size

- nothing

In [ ]:
help(im.set_size)

In [ ]:
im = Image()  # default is {'N_X':128, 'N_Y':128}
print(im.pe)

In [ ]:
im.set_size((512, 234))
print(im.pe)

In [ ]:
im.set_size(np.ones((512, 234)))
print(im.pe)

In [ ]:
im.set_size('http://upload.wikimedia.org/wikipedia/commons/d/de/Wikipedia_Logo_1.0.png')
print(im.pe)

## testing logging

we use https://docs.python.org/3.4/library/logging.html to handle logging of events

In [ ]:
im = Image()
print('Verbosity level=', im.pe.verbose)

In [ ]:
im.log.debug(' > this should not appear')
im.log.info(' > this should not appear')
im.log.error(' > this *should* appear')

In [ ]:
im.pe.verbose = 15
im.init_logging()
im.log.debug(' > this should not appear')
im.log.info(' > this *should* appear')
im.log.error(' > this *should* appear')

## basic parameter check

TODO

In [ ]:
try:
    Image({'N_X':128, 'N_Y':-12, 'mask_exponent':3})
except Exception as e:
    print(e) 

## testing reading an image file

Note that if the image is of a different size, the size of the ``Image`` object is adapted:

In [ ]:
im = Image()
print(im.pe)
image = im.imread('database/lena64.png')
print(im.pe)

In [ ]:
image = im.imread('http://upload.wikimedia.org/wikipedia/commons/d/de/Wikipedia_Logo_1.0.png')
print(im.pe)

## testing showing an image file



In [ ]:
'N_X' in im.pe.keys()


In [ ]:
im = Image()
print(im.pe)
image = im.imread('database/lena64.png')
_ = im.imshow(image)

In [ ]:
image = im.imread('http://upload.wikimedia.org/wikipedia/commons/d/de/Wikipedia_Logo_1.0.png')
_ = im.imshow(image)

In [ ]:
_ = im.show_spectrum(image)

## testing import and cropping

In [ ]:
axs = []
im = Image({'N_X':32, 'N_Y':128, 'seed':None, 'do_mask':False})
im.pe.datapath = 'database/'
for name_database in ['serre07_targets', 'serre07_distractors']:
    fig = plt.figure(figsize=(14, 3.5))
    for _ in range(4):
        image, filename, croparea = im.patch(name_database, )
        ax = fig.add_subplot(1, 4, _+1)
        im.imshow(image, fig=fig, ax=ax)
    fig.show()

## Handling a database

In [ ]:
im = Image({'N_X':64, 'N_Y':128, 'seed':None, 'do_mask':False, 'N_image':10})
im.pe.datapath = 'database/'
help(im.make_imagelist)
# extract one image list from a database
imagelist = im.make_imagelist('serre07_targets')
print('Number of images:', len(imagelist))
print('First image:', imagelist[0])

In [ ]:
help(im.get_imagelist)
im.pe.figpath, im.pe.matpath = '/tmp/fig', '/tmp/mat'
# store one image list from a database
imagelist = im.get_imagelist('classifier', 'serre07_targets')
!ls -l /tmp/mat
print('Number of images:', len(imagelist))
print('First image:', imagelist[0])

In [ ]:
im = Image({'N_X':128, 'N_Y':128, 'seed':None, 'do_mask':False, 'N_image':10})
im.pe.figpath, im.pe.matpath = '/tmp/fig', '/tmp/mat'
# now we can access again this stored list
!ls -l /tmp/mat
imagelist = im.get_imagelist('classifier', 'serre07_targets')
print('Number of images:', len(imagelist))
print('First image:', imagelist[0])

## checking basic translation mechanism

defining a reference test image; check the axis labels for a (x,y) translation

In [ ]:
im = Image({'N_X':128, 'N_Y':256})
image = np.zeros((im.pe.N_X, im.pe.N_Y))
image[im.pe.N_X//2:im.pe.N_X//2+im.pe.N_X//4, im.pe.N_X//2:im.pe.N_X//2+im.pe.N_X//4] = 1 # white square on the right
image[im.pe.N_X//2:im.pe.N_X//2+im.pe.N_X//4, im.pe.N_X//4:im.pe.N_X//2] = -1  # black square on its left
_ = im.imshow(image, axis=True)

translating the image by an integer by rolling indices


In [ ]:
print(im.pe.N_X//8., im.pe.N_Y//4.)
_ = im.imshow(np.roll(np.roll(image, np.int(im.pe.N_X//8.), axis=0), np.int(im.pe.N_X//4.), axis=1))

remember **axis conventions** are: x going down, y going right with the origin on top

translating the image down

In [ ]:
_ = im.imshow(im.translate(image, [im.pe.N_X//8., 0.]))

translating the image right

In [ ]:
_ = im.imshow(im.translate(image, [0., im.pe.N_Y//4.]))

translating the image on both axis

In [ ]:
_ = im.imshow(im.translate(image, [im.pe.N_X//8., im.pe.N_Y//4.]))

translating the image over the torus

In [ ]:
_ = im.imshow(im.translate(image, [im.pe.N_X//2., im.pe.N_Y//4.]))

[Back to top](#SLIP)

## some checks that should produce gray images (= null error)

null translation gives the same image

In [ ]:
_ = im.imshow(image - im.translate(image, [0., 0.]), norm=False)

the lg.translate function is invertible

In [ ]:
_ = im.imshow(image - im.translate(im.translate(image, [1.64, -2.233]), [-1.64, 2.233]), norm=False)

also true for bigger translations  (we separate integer part from fractional part in the translation)

In [ ]:
_ = im.imshow(image - im.translate(im.translate(image, [182.64, -286.233]), [-182.64, 286.233], preshift=True), norm=False)

but not always true when we don't separate

In [ ]:
_ = im.imshow(image - im.translate(im.translate(image, [182.64, -286.233]), [-182.64, 286.233], preshift=False), norm=False)

the lg.translate function is periodic

In [ ]:
_ = im.imshow(image - im.translate(image, [-45*im.pe.N_X, 5*im.pe.N_Y]), norm=False)

lg.translate function is associative

In [ ]:
_ = im.imshow(im.translate(image, [1., .0]) - im.translate(im.translate(image, [.5, .0]), [.5, .0]), norm=False)

In [ ]:
_ = im.imshow(im.translate(image, [2., .0]) - im.translate(im.translate(image, [1.5, -1.0]), [.5, 1.0]), norm=False)

## filtering

Check out the discussion @ http://blog.invibe.net/posts/2017-09-20-the-fastest-2d-convolution-in-the-world.html

## whitening

Here, we test the whitening on different images.

In [ ]:
axs = []
im = Image('default_param.py')
im.pe.datapath = 'database/'
for name_database in ['serre07_targets', 'serre07_distractors']:
    for _ in range(4):
        fig = plt.figure(figsize=(14, 7))
        image, filename, croparea = im.patch(name_database, )
        ax = fig.add_subplot(1, 2, 1)
        fig , ax = im.imshow(image, fig=fig, ax=ax)
        ax = fig.add_subplot(1, 2, 2)
        fig , ax = im.imshow(im.whitening(image), fig=fig, ax=ax)
        plt.show()

[Back to top](#SLIP)

## more book keeping

In [ ]:
%load_ext watermark
%watermark

[Back to top](#SLIP)